In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/best_chin_final.pth
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000304.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000050.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000280.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000286.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000182.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000111.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000038.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000292.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000327.jpg
/kaggle/input/equ

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

import numpy as np
import cv2
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from pathlib import Path
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class Config:
    BASE_DIR = Path("/kaggle/input/equine-exp3-minimal/equine_exp3_minimal")
    RIFE_CROPS_DIR = BASE_DIR / "cropped_regions_50FPS"
    OUTPUT_DIR = Path("/kaggle/working/final_results_exp3")
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    CLIP_LENGTH = 16
    CLASS_NAMES = ["Mild", "Moderate"]

    VIDEO_PAIN_LEVELS = {
        "S1_Video": 1, "S2_Video": 0, "S3_Video": 1, "S4_Video": 1,
        "S5_Video": 1, "S6_Video": 0, "S7_Video": 1, "S8_Video": 1,
        "S9_Video": 0, "S10_Video": 1, "S11_Video": 1, "S12_Video": 0,
    }

print("Device:", Config.DEVICE)
print("BASE_DIR:", Config.BASE_DIR)
print("Crops:", Config.RIFE_CROPS_DIR.exists())
print("Chin ckpt:", (Config.BASE_DIR / "best_chin_final.pth").exists())


Device: cuda
BASE_DIR: /kaggle/input/equine-exp3-minimal/equine_exp3_minimal
Crops: True
Chin ckpt: True


In [4]:
#  I3D model definition (Recreate the same I3D_EquinePain architecture used in training)
from torchvision.models.video import r3d_18

class I3D_EquinePain(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        backbone = r3d_18(pretrained=True)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # [B,512,1,1,1]
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: [B, C, T, H, W]
        feats = self.backbone(x)         # [B,512,1,1,1]
        feats = feats.view(feats.size(0), -1)  # [B,512]
        return self.classifier(feats)


In [5]:
# Chin I3D feature extractor (fixed checkpoint mapping)
from torchvision import transforms

class ChinI3DFeatureExtractor(nn.Module):
    """
    Loads trained chin I3D checkpoint and outputs 512-D features.
    FIXED for Kaggle: checkpoint key mapping is handled.
    """
    def __init__(self, ckpt_path, device=None):
        super().__init__()
        if device is None:
            device = Config.DEVICE
        self.device = device

        # Build I3D model
        model = I3D_EquinePain(num_classes=len(Config.CLASS_NAMES)).to(device)

        # Load checkpoint
        state = torch.load(ckpt_path, map_location=device)

        # ---- FIX: map "features." → "backbone.", "fc." → "classifier." ---- #
        raw_state = state["model_state_dict"] if "model_state_dict" in state else state
        new_state = {}

        for k, v in raw_state.items():
            nk = k

            if k.startswith("features."):
                nk = k.replace("features.", "backbone.")

            if k.startswith("fc."):
                nk = k.replace("fc.", "classifier.")

            new_state[nk] = v

        model.load_state_dict(new_state, strict=False)
        # ------------------------------------------------------------------ #

        model.eval()

        # extract backbone only (outputs 512-D vector)
        self.feature_extractor = nn.Sequential(*list(model.children())[:-1]).to(device)
        self.feature_extractor.eval()

        print(f"Loaded chin I3D feature extractor from {ckpt_path}")

    @torch.no_grad()
    def forward(self, clip):
        x = clip.unsqueeze(0).to(self.device)      # [1, C, 16, H, W]
        feat = self.feature_extractor(x)           # [1, 512] or [1,512,1,1,1]
        feat = feat.view(feat.size(0), -1)         # [1,512]
        return feat.squeeze(0).cpu()               # [512]

In [6]:
# Extract chin features per video
def extract_chin_i3d_features_for_videos(video_ids, split_name="all"):
    """
    For each video in video_ids, read chin frames and extract per-clip 512-D
    features. Saves: OUTPUT_DIR/i3d_chin_features/<video>.npy with shape [T_clips, 512].
    """
    ckpt_path = Config.BASE_DIR / "best_chin_final.pth"
    assert ckpt_path.exists(), f"Chin checkpoint not found: {ckpt_path}"

    extractor = ChinI3DFeatureExtractor(ckpt_path, device=Config.DEVICE)

    root = Config.RIFE_CROPS_DIR
    region = "chin"
    out_dir = Config.OUTPUT_DIR / "i3d_chin_features"
    out_dir.mkdir(parents=True, exist_ok=True)

    spatial_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(112),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]),
    ])

    print(f"\nEXTRACTING CHIN I3D FEATURES ({split_name}) → {out_dir}")

    for vid in video_ids:
        path = root / vid / region
        frames = sorted(path.glob("*.jpg"))
        if len(frames) < Config.CLIP_LENGTH:
            print(f"  {vid}: only {len(frames)} frames, skipping")
            continue

        clip_feats = []
        # non-overlapping 16-frame clips
        for i in range(0, len(frames) - Config.CLIP_LENGTH + 1, Config.CLIP_LENGTH):
            imgs = []
            for p in frames[i:i + Config.CLIP_LENGTH]:
                img = cv2.cvtColor(cv2.imread(str(p)), cv2.COLOR_BGR2RGB)
                img = spatial_transform(img)  # [C,H,W]
                imgs.append(img)
            clip = torch.stack(imgs, dim=1)      # [C,T,H,W]
            feat = extractor(clip)               # [512]
            clip_feats.append(feat.numpy())

        feats_arr = np.stack(clip_feats, axis=0)  # [num_clips,512]
        np.save(out_dir / f"{vid}.npy", feats_arr)
        print(f"  {vid}: {feats_arr.shape}")

    print("\nDone extracting chin features.")


Run this once to create features:

In [7]:
test_videos = ['S2_Video', 'S4_Video', 'S7_Video']
train_videos = [v for v in Config.VIDEO_PAIN_LEVELS if v not in test_videos]

extract_chin_i3d_features_for_videos(train_videos, split_name="train")
extract_chin_i3d_features_for_videos(test_videos,  split_name="test")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:00<00:00, 207MB/s]


Loaded chin I3D feature extractor from /kaggle/input/equine-exp3-minimal/equine_exp3_minimal/best_chin_final.pth

EXTRACTING CHIN I3D FEATURES (train) → /kaggle/working/final_results_exp3/i3d_chin_features
  S1_Video: (6, 512)
  S3_Video: (14, 512)
  S5_Video: (14, 512)
  S6_Video: (10, 512)
  S8_Video: (23, 512)
  S9_Video: (5, 512)
  S10_Video: (15, 512)
  S11_Video: (14, 512)
  S12_Video: (13, 512)

Done extracting chin features.
Loaded chin I3D feature extractor from /kaggle/input/equine-exp3-minimal/equine_exp3_minimal/best_chin_final.pth

EXTRACTING CHIN I3D FEATURES (test) → /kaggle/working/final_results_exp3/i3d_chin_features
  S2_Video: (2, 512)
  S4_Video: (22, 512)
  S7_Video: (22, 512)

Done extracting chin features.


In [8]:
# Sequence dataset + collate
class ChinI3DSequenceDataset(Dataset):
    def __init__(self, video_ids, labels):
        self.video_ids = video_ids
        self.labels = labels
        self.feature_dir = Config.OUTPUT_DIR / "i3d_chin_features"

        self.sequences = []
        self.seq_lengths = []

        for vid, lab in zip(video_ids, labels):
            path = self.feature_dir / f"{vid}.npy"
            feats = np.load(path)                   # [T_clips,512]
            self.sequences.append(torch.FloatTensor(feats))
            self.seq_lengths.append(len(feats))

        print(f"Loaded {len(self.sequences)} sequences "
              f"({min(self.seq_lengths)}–{max(self.seq_lengths)} clips).")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.seq_lengths[idx]


def collate_fn(batch):
    sequences, labels, lengths = zip(*batch)
    max_len = max(lengths)
    padded = []
    for seq in sequences:
        pad_len = max_len - len(seq)
        if pad_len > 0:
            pad = torch.zeros(pad_len, seq.size(1))
            padded.append(torch.cat([seq, pad], dim=0))
        else:
            padded.append(seq)
    padded = torch.stack(padded)          # [B,T,512]
    labels = torch.LongTensor(labels)
    lengths = torch.LongTensor(lengths)
    return padded, labels, lengths


In [9]:
# Bi‑LSTM model
class BiLSTMClassifier(nn.Module):
    def __init__(
        self,
        input_dim=512,
        hidden_dim=256,
        num_layers=2,
        num_classes=2,
        dropout=0.3,
        use_attention=True,
    ):
        super().__init__()
        self.use_attention = use_attention
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        if self.use_attention:
            self.attention = nn.Sequential(
                nn.Linear(hidden_dim * 2, hidden_dim),
                nn.Tanh(),
                nn.Linear(hidden_dim, 1),
            )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes),
        )

    def forward(self, x, lengths):
        # x: [B,T,512]
        B, T, _ = x.shape

        packed = nn.utils.rnn.pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        lstm_out, (h_n, _) = self.lstm(packed)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(
            lstm_out, batch_first=True, total_length=T
        )  # [B,T,2H]

        if self.use_attention:
            scores = self.attention(lstm_out)       # [B,T,1]
            mask = (torch.arange(T).unsqueeze(0).to(lengths.device) <
                    lengths.unsqueeze(1)).unsqueeze(2).float()
            scores = scores.masked_fill(mask == 0, -1e9)
            attn = torch.softmax(scores, dim=1)
            context = (lstm_out * attn).sum(dim=1)  # [B,2H]
        else:
            # last forward + last backward
            context = torch.cat([h_n[-2], h_n[-1]], dim=1)  # [B,2H]

        logits = self.classifier(context)           # [B,num_classes]
        return logits


In [10]:
# Train Bi‑LSTM
def train_bilstm(
    train_dataset,
    val_dataset,
    num_epochs=50,
    batch_size=4,
    lr=1e-3,
    device=None,
    save_path=None,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3,
    use_attention=True,
):
    if device is None:
        device = Config.DEVICE
    if save_path is None:
        save_path = Config.OUTPUT_DIR / "bilstm_chin_best.pth"

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        collate_fn=collate_fn, num_workers=2
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        collate_fn=collate_fn, num_workers=2
    )

    model = BiLSTMClassifier(
        input_dim=512,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=len(Config.CLASS_NAMES),
        dropout=dropout,
        use_attention=use_attention,
    ).to(device)

    train_labels = train_dataset.labels
    mild = sum(1 for l in train_labels if l == 0)
    mod  = sum(1 for l in train_labels if l == 1)
    weights = torch.FloatTensor([mod / max(mild,1), 1.0]).to(device)
    criterion = nn.CrossEntropyLoss(weight=weights)

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.5, patience=5, verbose=True
    )
    best_f1 = 0.0
    patience = 10
    patience_counter = 0

    history = {"train_loss": [], "val_loss": [], "val_f1": [], "val_acc": [], "lr": []}

    print("\n=== TRAINING CHIN I3D + BI-LSTM ===")

    for epoch in range(1, num_epochs+1):
        model.train()
        train_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")
        for x, y, lengths in pbar:
            x, y, lengths = x.to(device), y.to(device), lengths.to(device)
            optimizer.zero_grad()
            
            logits = model(x, lengths)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix({"loss": loss.item()})
        avg_train = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        val_preds, val_trues = [], []
        with torch.no_grad():
            for x, y, lengths in val_loader:
                x, y, lengths = x.to(device), y.to(device), lengths.to(device)
                logits = model(x, lengths)
                loss = criterion(logits, y)
                val_loss += loss.item()
                preds = torch.argmax(logits, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_trues.extend(y.cpu().numpy())
        avg_val = val_loss / len(val_loader)
        val_f1 = f1_score(val_trues, val_preds, average="macro")
        val_acc = np.mean(np.array(val_trues) == np.array(val_preds))

        history["train_loss"].append(avg_train)
        history["val_loss"].append(avg_val)
        history["val_f1"].append(val_f1)
        history["val_acc"].append(val_acc)
        history["lr"].append(optimizer.param_groups[0]["lr"])

        print(f"\nEpoch {epoch}: TrainLoss={avg_train:.4f}, "
              f"ValLoss={avg_val:.4f}, ValF1={val_f1:.4f}, ValAcc={val_acc:.3f}")

        scheduler.step(val_f1)

        if val_f1 > best_f1:
            best_f1 = val_f1
            patience_counter = 0
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "best_f1": best_f1,
                },
                save_path,
            )
            print(f"  New best model saved (F1={best_f1:.4f})")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("  Early stopping")
                break

    print(f"\nBest Val F1: {best_f1:.4f}")
    return model, best_f1, history


In [11]:
# Test evaluation
def evaluate_bilstm_on_test(
    model_path,
    test_dataset,
    device=None,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3,
    use_attention=True,
):
    if device is None:
        device = Config.DEVICE

    model = BiLSTMClassifier(
        input_dim=512,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=len(Config.CLASS_NAMES),
        dropout=dropout,
        use_attention=use_attention,
    ).to(device)

    ckpt = torch.load(model_path, map_location=device, weights_only=False)
    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()

    loader = DataLoader(
        test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn
    )

    all_preds, all_trues = [], []
    vids = test_dataset.video_ids

    print("\n=== CHIN I3D + BI-LSTM TEST EVAL ===")

    with torch.no_grad():
        for i, (x, y, lengths) in enumerate(loader):
            x, y, lengths = x.to(device), y.to(device), lengths.to(device)
            logits = model(x, lengths)
            probs = torch.softmax(logits, dim=1)
            pred = torch.argmax(probs, dim=1)

            all_preds.append(pred.item())
            all_trues.append(y.item())

            tname = Config.CLASS_NAMES[y.item()]
            pname = Config.CLASS_NAMES[pred.item()]
            mark = ">/" if y.item() == pred.item() else "X"
            print(f"{vids[i]:10s} | True={tname:9s} Pred={pname:9s} {mark}")

    test_f1 = f1_score(all_trues, all_preds, average="macro")
    test_acc = np.mean(np.array(all_trues) == np.array(all_preds))

    print("\nTest Accuracy:", test_acc)
    print("Test Macro-F1:", test_f1)
    print("\nReport:")
    print(classification_report(all_trues, all_preds,
                                target_names=Config.CLASS_NAMES, digits=3))
    print("Confusion matrix:")
    cm = confusion_matrix(all_trues, all_preds)
    print(cm)

    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=Config.CLASS_NAMES,
                yticklabels=Config.CLASS_NAMES)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Bi-LSTM (Chin) Confusion Matrix")
    plt.tight_layout()
    cm_path = Config.OUTPUT_DIR / f"confmat_{Path(model_path).stem}.png"
    plt.savefig(cm_path, dpi=150, bbox_inches="tight")
    plt.close()
    print("Saved confusion matrix to:", cm_path)

    return test_f1, test_acc


In [12]:
# Run Experiment 3 end‑to‑end
# Splits consistent with your earlier I3D experiments
test_videos = ['S2_Video', 'S4_Video', 'S7_Video']
train_videos = [v for v in Config.VIDEO_PAIN_LEVELS if v not in test_videos]

# Simple train/val split for Bi-LSTM
val_videos = train_videos[-2:]
train_v    = train_videos[:-2]

train_labels = [Config.VIDEO_PAIN_LEVELS[v] for v in train_v]
val_labels   = [Config.VIDEO_PAIN_LEVELS[v] for v in val_videos]
test_labels  = [Config.VIDEO_PAIN_LEVELS[v] for v in test_videos]

train_dataset = ChinI3DSequenceDataset(train_v, train_labels)
val_dataset   = ChinI3DSequenceDataset(val_videos, val_labels)
test_dataset  = ChinI3DSequenceDataset(test_videos, test_labels)

settings = [
    # name, use_attention, num_layers, hidden_dim, dropout
    ("base_attn_L2_H256", True,  2, 256, 0.3),  # your original setting
    ("no_attn_L2_H256",   False, 2, 256, 0.3),
    ("attn_L1_H256",      True,  1, 256, 0.3),
    ("attn_L2_H128",      True,  2, 128, 0.3),
    ("attn_L2_H256_do02", True,  2, 256, 0.2),
]

all_results = []

for name, use_attn, n_layers, h_dim, do in settings:
    print("\n" + "="*80)
    print(f"Running ablation setting: {name}")
    print("="*80)

    ckpt_path = Config.OUTPUT_DIR / f"bilstm_chin_{name}.pth"

    model, best_val_f1, hist = train_bilstm(
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        num_epochs=50,
        batch_size=4,
        lr=1e-3,
        device=Config.DEVICE,
        save_path=ckpt_path,
        hidden_dim=h_dim,
        num_layers=n_layers,
        dropout=do,
        use_attention=use_attn,
    )

    test_f1, test_acc = evaluate_bilstm_on_test(
        model_path=ckpt_path,
        test_dataset=test_dataset,
        device=Config.DEVICE,
        hidden_dim=h_dim,
        num_layers=n_layers,
        dropout=do,
        use_attention=use_attn,
    )

    all_results.append({
        "name": name,
        "use_attention": use_attn,
        "num_layers": n_layers,
        "hidden_dim": h_dim,
        "dropout": do,
        "val_f1": float(best_val_f1),
        "test_acc": float(test_acc),
        "test_f1": float(test_f1),
    })

    df_results = pd.DataFrame(all_results)
    csv_path = Config.OUTPUT_DIR / "bilstm_chin_ablation_results.csv"
    df_results.to_csv(csv_path, index=False)
    print("Saved ablation results to:", csv_path)
    df_results


print("\n=== ABLATION SUMMARY ===")
for r in all_results:
    print(r)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Loaded 7 sequences (5–23 clips).
Loaded 2 sequences (13–14 clips).
Loaded 3 sequences (2–22 clips).

Running ablation setting: base_attn_L2_H256

=== TRAINING CHIN I3D + BI-LSTM ===


Epoch 1/50: 100%|██████████| 2/2 [00:00<00:00,  2.43it/s, loss=0.656]



Epoch 1: TrainLoss=0.6755, ValLoss=0.7447, ValF1=0.3333, ValAcc=0.500
  New best model saved (F1=0.3333)


Epoch 2/50: 100%|██████████| 2/2 [00:00<00:00, 21.38it/s, loss=0.667]


...


Epoch 10: TrainLoss=0.5171, ValLoss=1.3023, ValF1=0.3333, ValAcc=0.500


Epoch 11/50: 100%|██████████| 2/2 [00:00<00:00, 21.53it/s, loss=0.49]



Epoch 11: TrainLoss=0.4473, ValLoss=1.5789, ValF1=0.3333, ValAcc=0.500
  Early stopping

Best Val F1: 0.3333


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== CHIN I3D + BI-LSTM TEST EVAL ===
S2_Video   | True=Mild      Pred=Moderate  X
S4_Video   | True=Moderate  Pred=Moderate  >/
S7_Video   | True=Moderate  Pred=Moderate  >/

Test Accuracy: 0.6666666666666666
Test Macro-F1: 0.4

Report:
              precision    recall  f1-score   support

        Mild      0.000     0.000     0.000         1
    Moderate      0.667     1.000     0.800         2

    accuracy                          0.667         3
   macro avg      0.333     0.500     0.400         3
weighted avg      0.444     0.667     0.533         3

Confusion matrix:
[[0 1]
 [0 2]]
Saved confusion matrix to: /kaggle/working/final_results_exp3/confmat_bilstm_chin_attn_L2_H256_do02.png
Saved ablation results to: /kaggle/working/final_results_exp3/bilstm_chin_ablation_results.csv

=== ABLATION SUMMARY ===
{'name': 'base_attn_L2_H256', 'use_attention': True, 'num_layers': 2, 'hidden_dim': 256, 'dropout': 0.3, 'val_f1': 0.3333333333333333, 'test_acc': 0.6666666666666666, 'test_f1':

In [13]:
%cd /kaggle/working

!zip -r exp3_bilstm_outputs.zip final_results_exp3
!ls -lh

/kaggle/working
  adding: final_results_exp3/ (stored 0%)
  adding: final_results_exp3/confmat_bilstm_chin_base_attn_L2_H256.png (deflated 11%)
  adding: final_results_exp3/bilstm_chin_attn_L2_H128.pth (deflated 9%)
  adding: final_results_exp3/bilstm_chin_attn_L2_H256_do02.pth (deflated 8%)
  adding: final_results_exp3/confmat_bilstm_chin_attn_L1_H256.png (deflated 11%)
  adding: final_results_exp3/confmat_bilstm_chin_attn_L2_H128.png (deflated 11%)
  adding: final_results_exp3/confmat_bilstm_chin_attn_L2_H256_do02.png (deflated 11%)
  adding: final_results_exp3/bilstm_chin_ablation_results.csv (deflated 66%)
  adding: final_results_exp3/bilstm_chin_base_attn_L2_H256.pth (deflated 8%)
  adding: final_results_exp3/i3d_chin_features/ (stored 0%)
  adding: final_results_exp3/i3d_chin_features/S3_Video.npy (deflated 10%)
  adding: final_results_exp3/i3d_chin_features/S8_Video.npy (deflated 10%)
  adding: final_results_exp3/i3d_chin_features/S7_Video.npy (deflated 10%)
  adding: final_resu